<a href="https://colab.research.google.com/github/pmxfa/sp-shapely/blob/main/sp_timegan_exchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install synthcity tsbootstrap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.2/552.2 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchtext to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is stil

# Training

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader
from synthcity.utils.serialization import save_to_file, load_from_file

log.add(sink=sys.stderr, level="INFO")

Mounted at /content/drive
[KeOps] Warning : CUDA libraries not found or could not be loaded; Switching to CPU only.


In [2]:
file_path = "/content/drive/Shareddrives/sp_env/datasets/Exchange Rate/exchange_rate.txt"

df = pd.read_csv(file_path)
print(df.head())
print(df.info())
print('missing values:', df.isnull().sum())

   0.785500  1.611000  0.861698  0.634196  0.211242  0.006838  0.593000  \
0    0.7818    1.6100  0.861104  0.633513  0.211242  0.006863    0.5940   
1    0.7867    1.6293  0.861030  0.648508  0.211242  0.006975    0.5973   
2    0.7860    1.6370  0.862069  0.650618  0.211242  0.006953    0.5970   
3    0.7849    1.6530  0.861995  0.656254  0.211242  0.006940    0.5985   
4    0.7866    1.6537  0.861030  0.654879  0.211242  0.006887    0.6040   

   0.525486  
0  0.523972  
1  0.526316  
2  0.523834  
3  0.527426  
4  0.526177  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7587 entries, 0 to 7586
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   0.785500  7587 non-null   float64
 1   1.611000  7587 non-null   float64
 2   0.861698  7587 non-null   float64
 3   0.634196  7587 non-null   float64
 4   0.211242  7587 non-null   float64
 5   0.006838  7587 non-null   float64
 6   0.593000  7587 non-null   float64
 7   0.5

In [3]:
# for exchange dataset only
df.columns = [
    'Australia', 'British_Pound', 'Canada', 'Switzerland',
    'China', 'Japan', 'New_Zealand', 'Singapore'
]

# set seed and randomly choose 2 features
np.random.seed(42)

# Filter the dataframe to only include selected features
selected_features = np.random.choice(df.columns, size=2, replace=False)
print(f"Selected features: {selected_features}")

# Immediately filter your full DataFrame down to just those two series:
df = df[selected_features]

Selected features: ['British_Pound' 'Japan']


In [4]:
# Keep the latest 5000 rows
df_latest = df.tail(5000)

# Train-test split: 70% for training (for TimeGAN), 30% for testing (TSTR)
train_size = int(0.7 * len(df_latest))
df_train = df_latest.iloc[:train_size]
df_test = df_latest.iloc[train_size:]  # use later for LSTM-TSTR

# Normalize the data
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df_train)
df_scaled_train = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)
scaled_test = scaler.transform(df_test)
df_scaled_test = pd.DataFrame(scaled_test, columns=df_test.columns, index=df_test.index)


# set sequence length to 30 to capture monthly patterns
# dataset = daily
sequence_length = 30
# temporal_data = []
# observation_times = []

# for start in range(len(df_scaled_train) - sequence_length + 1):
#     sequence = df_scaled_train.iloc[start:start + sequence_length].reset_index(drop=True)
#     temporal_data.append(sequence)
#     observation_times.append(list(range(sequence_length)))  # relative time within the window


# # Dummy outcome for data loader
# dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data)), columns=["outcome"])

# # --- Create DataLoader for TimeGAN ---
# loader = TimeSeriesDataLoader(
#     temporal_data=temporal_data,  # List of sequences (DataFrames)
#     observation_times=observation_times,  # List of time indices (DataFrames)
#     static_data=None,  # No static data for now (can be set if needed)
#     outcome=dummy_outcome,  # Dummy outcome for forecasting
# )

# # Print the loader info
# print(f"TimeSeriesDataLoader created with {len(temporal_data)} sequences")

In [5]:
temporal_data_test = []
observation_times_test = []

# Generate sequences from df_scaled_test only
for start in range(len(df_scaled_test) - sequence_length + 1):
    sequence = df_scaled_test.iloc[start:start + sequence_length].reset_index(drop=True)
    temporal_data_test.append(sequence)
    observation_times_test.append(list(range(sequence_length)))  # relative time within the window

# Dummy outcome for TimeGAN (can be used in DataLoader)
dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data_test)), columns=["outcome"])

# Create DataLoader for TimeGAN
loader_test = TimeSeriesDataLoader(
    temporal_data=temporal_data_test,
    observation_times=observation_times_test,
    static_data=None,
    outcome=dummy_outcome,
)

# Print the loader info
print(f"TimeSeriesDataLoader TEST SET created with {len(temporal_data_test)} sequences")

TimeSeriesDataLoader TEST SET created with 1471 sequences


In [ ]:
print(len(df_train))  # Check the length of the dataframe
print(loader.dataframe())

3500
        seq_id  seq_time_id  seq_temporal_British_Pound  seq_temporal_Japan  \
0            0            0                    0.175634            0.327109   
1            0            1                    0.177668            0.321058   
2            0            2                    0.168709            0.315871   
3            0            3                    0.163281            0.314661   
4            0            4                    0.165788            0.318638   
...        ...          ...                         ...                 ...   
104125    3470           25                    0.325966            0.882434   
104126    3470           26                    0.327503            0.875692   
104127    3470           27                    0.313072            0.865318   
104128    3470           28                    0.312271            0.866010   
104129    3470           29                    0.305651            0.874308   

        seq_out_outcome  
0                   

In [ ]:
from synthcity.plugins import Plugins

hparams = {
          "mode": "LSTM",
}

# Load TimeGAN with custom parameters
syn_model = Plugins().get("timegan", **hparams)

[2025-04-26T13:20:23.621941+0000][2010][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2025-04-26T13:20:23.621941+0000][2010][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py


In [ ]:
# Print all parameters of initialized model
for attr in dir(syn_model):
    if not attr.startswith("_") and not callable(getattr(syn_model, attr)):
        print(f"{attr}: {getattr(syn_model, attr)}")

batch_size: 64
class_name: TimeGANPlugin
clipping_value: 0
compress_dataset: False
dataloader_sampling_strategy: imbalanced_time_censoring
device: cuda
discriminator_batch_norm: False
discriminator_dropout: 0.1
discriminator_loss: None
discriminator_lr: 0.001
discriminator_n_iter: 1
discriminator_n_layers_hidden: 3
discriminator_n_units_hidden: 300
discriminator_nonlin: leaky_relu
discriminator_weight_decay: 0.001
embedding_penalty: 10
encoder: None
encoder_max_clusters: 20
expecting_conditional: False
fitted: False
gamma_penalty: 1
generator_batch_norm: False
generator_dropout: 0.01
generator_loss: None
generator_lr: 0.001
generator_n_layers_hidden: 2
generator_n_units_hidden: 150
generator_nonlin: leaky_relu
generator_nonlin_out_continuous: tanh
generator_nonlin_out_discrete: softmax
generator_residual: True
generator_weight_decay: 0.001
mode: LSTM
module_name: synthcity.plugins.time_series.plugin_timegan
module_relative_path: ../time_series/plugin_timegan.py
moments_penalty: 100
n_i

## fitting the model

In [ ]:
print(loader.shape)

# --- Train the model ---
syn_model.fit(loader)

(104130, 5)


100%|██████████| 1000/1000 [1:56:52<00:00,  7.01s/it]


In [ ]:
#@title optional: generate data from saved model
from synthcity.utils.serialization import save_to_file, load_from_file

# Save model to drive
save_to_file('/content/drive/Shareddrives/sp_env/saved_models/GAN_Exchange.pkl', syn_model)

# Load the model
# syn_model = load_from_file('/content/drive/Shareddrives/sp_env/synmodel.pkl')

In [ ]:
# --- Generate Synthetic Data ---
n_samples = len(temporal_data)
syn_data = syn_model.generate(count=n_samples)
print(syn_data.shape)

(100017, 5)


In [ ]:
# --- Save with automated format ---
import datetime
import os
# Get the current date and time
now = datetime.datetime.now()
timestamp = now.strftime("%m%d%y-%H%M%S")  # MMDDYY-HHMMSS format

# Define the base directory
base_dir = "/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/exchange"  #CHANGE THIS
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Construct the filename
model_name = type(syn_model).__name__.lower() # Get model name dynamically
filename = f"{timestamp}-{model_name}-3500.csv"
filepath = os.path.join(base_dir, filename)

# Save the data
df_syn = syn_data.dataframe()
df_syn.to_csv(filepath, index=False)

print(f"Synthetic data saved to: {filepath}")

Synthetic data saved to: /content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/exchange/042625-152949-timeganplugin-3500.csv


# Evaluation

## Prerequisites

In [6]:
syn_data = pd.read_csv('/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/exchange/042625-152949-timeganplugin-3500.csv')

In [9]:
# Define selected columns explicitly
# selected_columns = ['seq_temporal_HUFL', 'seq_temporal_HULL', 'seq_temporal_LUFL', 'seq_temporal_LULL', 'seq_temporal_MUFL', 'seq_temporal_MULL', 'seq_temporal_OT']
selected_columns = ['seq_temporal_British_Pound','seq_temporal_Japan']
# Ensure real_data and synthetic_data only contain the selected columns
real_data = loader_test.dataframe()[selected_columns].to_numpy()
synthetic_data = syn_data[selected_columns].to_numpy()

In [10]:
print(real_data, "\n ------------------------------------------------------- \n", synthetic_data)
print(type(real_data),type(synthetic_data))
print(real_data.shape,synthetic_data.shape)

[[ 0.30750644  0.87102351]
 [ 0.3059454   0.88191563]
 [ 0.3059454   0.88520055]
 ...
 [-0.18759935  0.197787  ]
 [-0.18787898  0.19657676]
 [-0.18787898  0.19657676]] 
 ------------------------------------------------------- 
 [[0.25763111 0.18547966]
 [0.0866552  0.18546773]
 [0.08670829 0.18559386]
 ...
 [0.31743031 0.1863488 ]
 [0.25738601 0.18644113]
 [0.61835132 0.18653295]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(44130, 2) (100017, 2)


## Generate distance metrics

### Helper Functions

In [11]:
from scipy.stats import wasserstein_distance, entropy
import numpy as np

def compute_wasserstein(real_data, synthetic_data, selected_columns):
    """
    Computes Wasserstein Distance between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order (no random sampling)
    synthetic_trimmed = synthetic_data[:min_length]  # Match size
    print(real_trimmed.shape,synthetic_trimmed.shape)

    wasserstein_results = {}

    # Compute Wasserstein Distance for each feature
    for i, col in enumerate(selected_columns):
        w_dist = wasserstein_distance(real_trimmed[:, i], synthetic_trimmed[:, i])
        wasserstein_results[col] = w_dist
        print(f"{w_dist}")

    return wasserstein_results

def compute_kl_divergence(real_data, synthetic_data, selected_columns, bins=50):
    """
    Computes KL Divergence between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order
    synthetic_trimmed = synthetic_data[:min_length]  # Match size

    kl_results = {}

    for i, col in enumerate(selected_columns):
        # Compute histogram-based probability distributions
        real_hist, _ = np.histogram(real_trimmed[:, i], bins=bins, density=True)
        synth_hist, _ = np.histogram(synthetic_trimmed[:, i], bins=bins, density=True)

        # Avoid zero probabilities (KL Divergence is undefined for zero values)
        real_hist += 1e-10
        synth_hist += 1e-10

        # Compute KL Divergence
        kl_div = entropy(real_hist, synth_hist)
        kl_results[col] = kl_div
        print(f"{kl_div}")

    return kl_results

### Generate Metrics

In [12]:
# Assuming df_scaled is the DataFrame containing your scaled ETD data

# Compute Wasserstein Distance
wasserstein_results = compute_wasserstein(real_data, synthetic_data, selected_columns)
print("Wasserstein Distance Results:")
print(wasserstein_results)

# Compute KL Divergence
kl_results = compute_kl_divergence(real_data, synthetic_data, selected_columns)
print("KL Divergence Results:")
print(kl_results)

(44130, 2) (44130, 2)
0.08117466787512753
0.15334526304995016
Wasserstein Distance Results:
{'seq_temporal_British_Pound': 0.08117466787512753, 'seq_temporal_Japan': 0.15334526304995016}
17.43605457568381
17.20093945765117
KL Divergence Results:
{'seq_temporal_British_Pound': 17.43605457568381, 'seq_temporal_Japan': 17.20093945765117}


# LSTM downstream

In [ ]:
real_data = loader_test.dataframe()
df_synth = pd.read_csv("/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/exchange/042625-152949-timeganplugin-3500.csv")

# drop unwanted column
real_data = real_data.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")
df_synth = df_synth.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")

print(f"real_data: {real_data.shape}, synthetic_data: {df_synth.shape}")

real_data: (3500, 2), synthetic_data: (100017, 2)


In [17]:
real_data = loader_test.dataframe()
df_synth = syn_data

# 2. Drop the unwanted column
real_data = real_data.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")
df_synth = df_synth.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")

In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [19]:
# Convert to tensors (float32 for PyTorch)
data_real = torch.tensor(real_data.values, dtype=torch.float32)
data_synth = torch.tensor(df_synth.values, dtype=torch.float32)

#  Sequence builder
def make_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return torch.stack(X), torch.stack(y)

SEQ_LEN = sequence_length

# Sequences for synthetic (train)
X_train, y_train = make_sequences(data_synth, SEQ_LEN)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)

# Sequences for real (test)
X_test, y_test = make_sequences(data_real, SEQ_LEN)

In [20]:

# ─── Model Definition ──────────────────────────────────────
class ShallowLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)  # hn shape: (1, batch, hidden_size)
        out = self.linear(hn.squeeze(0))  # squeeze to (batch, hidden_size)
        return out


# ─── Model Init ─────────────────────────────────────────────
model = ShallowLSTM(input_size=X_train.shape[2], hidden_size=64)

# ─── Optimizer & Loss ───────────────────────────────────────
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [ ]:
# ─── Training ───────────────────────────────────────────────
EPOCHS = 50
for epoch in range(1, EPOCHS + 1):
    model.train()
    for xb, yb in train_loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # if epoch % 10 == 0 or epoch == 1:
    print(f"Epoch {epoch}: Train MSE = {loss.item():.6f}")

In [ ]:
#@title ✧.* model evaluation ✧.*
model.eval()
with torch.no_grad():
    preds = model(X_test)
    test_mse = loss_fn(preds, y_test).item()
    test_mae = mean_absolute_error(y_test.numpy(), preds.numpy())

    print(f"Test MSE: {test_mse:.6f}")
    print(f"Test MAE: {test_mae:.6f}")

Test MSE: 0.103023
Test MAE: 0.235100


# Bootstrapping

In [22]:
from tsbootstrap import MovingBlockBootstrap
import numpy as np

bootstrap_configs = {
    "weather": {"block_length": 36, "n_bootstraps": 15, "rng": 42},       # 6-hour pattern (10-min interval)
    "electricity": {"block_length": 24, "n_bootstraps": 15, "rng": 42},   # 1-day pattern (hourly)
    "exchange": {"block_length": 30, "n_bootstraps": 15, "rng": 42},      # 1-month pattern (daily)
}

# Example for weather
dataset_name = "exchange"
config = bootstrap_configs[dataset_name]

real_test_array = real_data[selected_columns].to_numpy()  # shape (N, features)
mbb = MovingBlockBootstrap(
    n_bootstraps=config["n_bootstraps"],
    rng=config["rng"],
    block_length=config["block_length"]
)
boot_samples = mbb.bootstrap(real_test_array, return_indices=False)

In [23]:
bootstrap_results = []

for b_idx, boot_real in enumerate(boot_samples):
    # 1. Match the synthetic data size
    syn_trimmed = synthetic_data[:len(boot_real)]

    # 2. Fidelity metrics
    wasserstein = compute_wasserstein(boot_real, syn_trimmed, selected_columns)
    kl = compute_kl_divergence(boot_real, syn_trimmed, selected_columns)

    # 3. Utility metrics
    # Preprocess this bootstrap sample for LSTM (as you do with real_data)
    boot_tensor = torch.tensor(boot_real, dtype=torch.float32)
    Xb_test, yb_test = make_sequences(boot_tensor, SEQ_LEN)

    model.eval()
    with torch.no_grad():
        preds = model(Xb_test)
        mse = mean_squared_error(yb_test.numpy(), preds.numpy())
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(yb_test.numpy(), preds.numpy())

    # 4. Store results
    bootstrap_results.append({
        'bootstrap': b_idx,
        'wasserstein': np.mean(list(wasserstein.values())),
        'kl': np.mean(list(kl.values())),
        'rmse': rmse,
        'mae': mae
    })

(44130, 2) (44130, 2)
0.0830805148889528
0.1536210888468385
17.22113952016976
17.17537103841086
(44130, 2) (44130, 2)
0.08140693101833461
0.151777377023289
17.570698564722292
17.093488783171985
(44130, 2) (44130, 2)
0.07904870100812919
0.15336695594370459
17.404968717813574
17.111706387512072
(44130, 2) (44130, 2)
0.07371995443513063
0.1550851659946605
17.604149779229555
17.497905603480316
(44130, 2) (44130, 2)
0.0783790714929946
0.1528026503834652
17.55428492841301
17.18087286787068
(44130, 2) (44130, 2)
0.08656598937119925
0.15277463234966976
17.150676311522567
17.387680415791646
(44130, 2) (44130, 2)
0.07948165084286721
0.1551900434558216
17.464554902022062
17.39558241925526
(44130, 2) (44130, 2)
0.08018590411887444
0.14861175137008495
17.500306343573524
17.070677191868725
(44130, 2) (44130, 2)
0.07661886504049417
0.15364064790638426
17.540829620098332
17.46420603818756
(44130, 2) (44130, 2)
0.08574885338852964
0.14794079273658156
17.18785140096761
17.136194500628374
(44130, 2) (441

In [24]:
print(bootstrap_results)

# Assuming bootstrap_results is your list of dicts
df_results = pd.DataFrame(bootstrap_results)

df_results['Dataset'] = 'Ex'
df_results['Model'] = 'GAN'

df_results

[{'bootstrap': 0, 'wasserstein': 0.11835080186789565, 'kl': 17.198255279290308, 'rmse': 0.38150477, 'mae': 0.34895927}, {'bootstrap': 1, 'wasserstein': 0.1165921540208118, 'kl': 17.33209367394714, 'rmse': 0.38105005, 'mae': 0.3488917}, {'bootstrap': 2, 'wasserstein': 0.11620782847591689, 'kl': 17.258337552662823, 'rmse': 0.3816973, 'mae': 0.3492574}, {'bootstrap': 3, 'wasserstein': 0.11440256021489556, 'kl': 17.551027691354935, 'rmse': 0.38413608, 'mae': 0.35278928}, {'bootstrap': 4, 'wasserstein': 0.1155908609382299, 'kl': 17.367578898141844, 'rmse': 0.38333693, 'mae': 0.35128868}, {'bootstrap': 5, 'wasserstein': 0.1196703108604345, 'kl': 17.269178363657105, 'rmse': 0.3816743, 'mae': 0.3482936}, {'bootstrap': 6, 'wasserstein': 0.1173358471493444, 'kl': 17.43006866063866, 'rmse': 0.38431364, 'mae': 0.35078558}, {'bootstrap': 7, 'wasserstein': 0.1143988277444797, 'kl': 17.285491767721126, 'rmse': 0.37837842, 'mae': 0.347053}, {'bootstrap': 8, 'wasserstein': 0.11512975647343922, 'kl': 17

,bootstrap,wasserstein,kl,rmse,mae,Dataset,Model
0,0,0.118351,17.198255,0.381505,0.348959,Ex,GAN
1,1,0.116592,17.332094,0.381050,0.348892,Ex,GAN
2,2,0.116208,17.258338,0.381697,0.349257,Ex,GAN
3,3,0.114403,17.551028,0.384136,0.352789,Ex,GAN
4,4,0.115591,17.367579,0.383337,0.351289,Ex,GAN
5,5,0.119670,17.269178,0.381674,0.348294,Ex,GAN
6,6,0.117336,17.430069,0.384314,0.350786,Ex,GAN
7,7,0.114399,17.285492,0.378378,0.347053,Ex,GAN
8,8,0.115130,17.502518,0.382613,0.350802,Ex,GAN
9,9,0.116845,17.162023,0.376687,0.344115,Ex,GAN


In [25]:
summary_row = {
    'Dataset': 'El',
    'Model': 'GAN',
    'Wasserstein': df_results['wasserstein'].mean(),
    'KL': df_results['kl'].mean(),
    'RMSE': df_results['rmse'].mean(),
    'MAE': df_results['mae'].mean()
}

df_summary = pd.DataFrame([summary_row])
print(df_summary)

  Dataset Model  Wasserstein         KL      RMSE      MAE
0      El   GAN      0.11724  17.295677  0.381286  0.34857
